In [49]:
from myosuite.utils import gym

In [50]:
import mujoco
env = gym.make('myoHandPoseRandom-v0', normalize_act = False)

In [51]:
import numpy as np
env.env.init_qpos[:] = np.zeros(len(env.env.init_qpos),)
mjcModel = env.env.sim.model

In [52]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 400):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [53]:
# 오프스크린 프레임버퍼 크기 설정
mjcModel.vis.global_.offwidth = 1920
mjcModel.vis.global_.offheight = 1080

In [54]:
body_names = [mjcModel.id2name(i, mujoco.mjtObj.mjOBJ_BODY) for i in range(mjcModel.nbody)]
body_names

['world',
 'full_body',
 'thorax',
 'clavicle',
 'clavphant',
 'scapula',
 'scapphant',
 'humphant',
 'humphant1',
 'humerus',
 'ulna',
 'radius',
 'lunate',
 'scaphoid',
 'pisiform',
 'triquetrum',
 'capitate',
 'trapezium',
 'firstmc',
 'proximal_thumb',
 'distal_thumb',
 'trapezoid',
 'hamate',
 'secondmc',
 'proxph2',
 'midph2',
 'distph2',
 'thirdmc',
 'proxph3',
 'midph3',
 'distph3',
 'fourthmc',
 'proxph4',
 'midph4',
 'distph4',
 'fifthmc',
 'proxph5',
 'midph5',
 'distph5']

In [55]:
key_frame_interval = 30
video_fps = 30
# steps_per_frame = int(1/(mjcModel.opt.timestep * video_fps))
steps_per_frame = 1 / (mjcModel.opt.timestep * video_fps) # 0.002(1/500), 30fps이면 1초 500step인데 이것을 30으로 나누면 16.666step에 1frame
# 1 / (sec/step * frame / sec) = step / frame
playback_duration = 10
total_step_count = round(playback_duration / mjcModel.opt.timestep)
frames = []

env.reset()
next_frame_step = 0
for step_idx in range(total_step_count):
    # if not(step_idx%steps_per_frame):
    # if (step_idx * video_fps * mjcModel.opt.timestep) % 1 < mjcModel.opt.timestep:
    if step_idx >= next_frame_step:
        frame = env.sim.renderer.render_offscreen(
                                    width=1920,
                                    height=1080,
                                    camera_id=3)
        frames.append(frame)
        next_frame_step += steps_per_frame

    ctrl = np.zeros(mjcModel.na,)
    # ctrl[mjcModel.actuator('FDP2').id] = 0.2
    ctrl[mjcModel.actuator('EDC2').id] = 0.2

    env.step(ctrl)
print(f"total_step_count: {total_step_count}, len(frames): {len(frames)}")
import os
import skvideo.io

os.makedirs('temp_videos', exist_ok=True)
# make a local copy
output_dict = {
    "-pix_fmt": "yuv420p",
    "-r": f"{video_fps}",
    "-g": f"{key_frame_interval}",
    "-video_track_timescale": "30000",
    "-b:v": "1M",
    "-bufsize": "2M" ,
    # "-map": "0",
    # "-vcodec": "libx264",
    # "-f": "mpegts"
}
skvideo.io.vwrite('temp_videos/myosuite_study_01.mp4', np.asarray(frames),outputdict=output_dict)

# show in the notebook
show_video('temp_videos/myosuite_study_01.mp4', video_width=1920)

total_step_count: 5000, len(frames): 300


In [23]:
f"{video_fps}"

'30'